# 옵저버 패턴(Observer Pattern)

- 참고: [헤드 퍼스트 디자인 패턴](https://ebook-product.kyobobook.co.kr/dig/epd/ebook/E000002942608) Chapter2. 객체들에게 연락 돌리기(옵저버 패턴)

# 1. 주요 내용
- 옵저버 패턴이란?
    - 한 객체의 상태가 변하면 그 객체에 의존하는 객체에게 연락이 가고, 자동으로 내용이 갱신되는 방식
    - 일대다 관계를 가진다.
    - 주제(Subject)는 동일한 인터페이스로 옵저버에게 연락한다.
    - 주제가 데이터를 보내는 `Push 방식`과 옵저버가 데이터를 가져오는 `Pull 방식`이 있다. 일반적으로 Pull 방식이 더 옳다고 여겨진다.

- 느슨한 결합(loose coupling)
    - 단단한 바구니가 유연한 바구니보다 부서지기 쉽듯, 소프트웨어도 한 객체가 다른 객체에 심하게 의존하면 부서지기 쉽다.
    - 객체가 느슨하게 결합되어 있다는 말의 의미는 의존은 하되 **다른 객체의 세세한 부분까지 다 알 필요가 없다는** 의미다.
    - 옵저버 패턴은 느슨한 결합의 훌륭한 예시다.

# 2. TO-DO
- 상기 책에서 설명하는 옵저버 패턴 예시를 파이썬으로 구현해보기
- Subject: 기상 스테이션으로부터 받는 데이터를 추적하는 객체
- Observer: 위 객체에 의존하여 필요한 데이터를 받아 사용하는 객체

In [1]:
from abc import *

In [2]:
# 주제(Subject) 인터페이스
class Subject(metaclass=ABCMeta):
    @abstractmethod
    def register_observer():
        pass
    
    @abstractmethod
    def remove_observer():
        pass
    
    @abstractmethod
    def notify_observer():
        pass


# 옵저버(Observer) 인터페이스
class Observer(metaclass=ABCMeta):
    @abstractmethod
    def update():
        pass


# 디스플레이 인터페이스
class DisplayElement(metaclass=ABCMeta):
    @abstractmethod
    def display():
        pass

In [3]:
# Subject 구현 클래스
class WeatherData(Subject):
    def __init__(self):
        self.observers = []
        self.temperature = 0
        self.humidity = 0
        self.pressure = 0
        
    def register_observer(self, observer:Observer):
        if observer in self.observers:
            print('WeatherData | 이미 등록된 observer입니다.')
            return
        
        self.observers.append(observer)
        print(f'WeatherData | {observer}를 성공적으로 등록했습니다.')
    
    def remove_observer(self, observer:Observer):
        if observer not in self.observers:
            print('WeatherData | 등록되지 않은 Observer입니다.')
            return
        
        self.observers.remove(observer)
        print(f'WeatherData | {observer}를 성공적으로 삭제했습니다.')
    
    def notify_observer(self):
        for observer in self.observers:
            observer.update()
    
    def get_temperature(self):
        return self.temperature
    
    def get_humidity(self):
        return self.humidity
    
    def get_pressure(self):
        return self.pressure
    
    def measurements_changed(self):
        self.notify_observer()
    
    # 이 메소드를 통해 기상 스테이션으로부터 새로운 데이터를 받는다고 가정
    def set_weather_data(self, temperature:float=None, humidity:float=None, pressure:float=None):
        self.temperature = temperature
        self.humidity = humidity
        self.pressure = pressure
        self.measurements_changed()
        print('WeatherData | 기상 데이터가 갱신되었습니다.')

In [4]:
# Observer 구현 클래스
# 기온 데이터의 최솟값, 최댓값, 평균값을 보여주는 디스플레이
class StatisticsDisplay(Observer, DisplayElement):
    def __init__(self, weather_data:WeatherData):
        self.weather_data = weather_data
        self.temperatures = []
        
        # 인스턴스 생성 시 바로 Observer 등록
        self.start_observing()
        
    def update(self):
        # 필요한 정보인 기온 정보만 가져온다.
        self.temperatures.append(self.weather_data.get_temperature())
        print('StatisticsDisplay | 기온 데이터를 가져왔습니다.')
    
    def display(self):
        content = f'''
        최대 기온: {max(self.temperatures)}
        최소 기온: {min(self.temperatures)}
        평균 기온: {sum(self.temperatures) / len(self.temperatures)}
        '''
        return content
    
    def start_observing(self):
        self.weather_data.register_observer(self)

    def stop_observing(self):
        self.weather_data.remove_observer(self)
        
    def __repr__(self):
        return '통계 디스플레이'

In [5]:
# WeatherData 인스턴스 만들기
weather_data = WeatherData()

# StatisticsDisplay 인스턴스 만들기
statistics_display = StatisticsDisplay(weather_data)

WeatherData | 통계 디스플레이를 성공적으로 등록했습니다.


In [6]:
# 기상 데이터 갱신하기
weather_data.set_weather_data(temperature=30)
weather_data.set_weather_data(temperature=40)
weather_data.set_weather_data(temperature=20)
weather_data.set_weather_data(temperature=15)

StatisticsDisplay | 기온 데이터를 가져왔습니다.
WeatherData | 기상 데이터가 갱신되었습니다.
StatisticsDisplay | 기온 데이터를 가져왔습니다.
WeatherData | 기상 데이터가 갱신되었습니다.
StatisticsDisplay | 기온 데이터를 가져왔습니다.
WeatherData | 기상 데이터가 갱신되었습니다.
StatisticsDisplay | 기온 데이터를 가져왔습니다.
WeatherData | 기상 데이터가 갱신되었습니다.


In [7]:
# 갱신된 데이터를 받아 제대로 출력하는지 확인
print(statistics_display.display())


        최대 기온: 40
        최소 기온: 15
        평균 기온: 26.25
        
